In [ ]:
import tools.create_masks as masker
import tools.image_resizer as resizer
import tools.image_annotator as annotator
import tools.image_tiles as tiler

# pip install opencv-python
#apt-get install ffmpeg libsm6 libxext6  -y



In [ ]:
import tensorflow as tf; print(tf.config.list_physical_devices('GPU'))

In [ ]:
masker.create_masks_for_folder("data/source-images", "data/source-annotations", "data/source-masks")

In [ ]:
annotator.create_image_annotations_for_folder("data/source-images", "data/source-annotations", "data/source-images-annotated")

In [ ]:
resizer.resize_images_in_folder("data/source-images", "data/images", (5000, 5000))

In [ ]:
resizer.resize_images_in_folder("data/source-masks", "data/masks", (5000, 5000))

In [ ]:
resizer.resize_images_in_folder("data/source-images-annotated", "data/images-annotated", (5000, 5000))

In [ ]:
tiler.split_to_tiles("data/source-images", "data/image-tiles", "data/source-masks", "data/mask-tiles", 256, 256 ,"data/visualized_masks/tiles.csv", "")

In [ ]:
#tiler.split_image_into_tiles_with_background("data/images/1.jpg", "data/test_tiles", "data/masks/1.png", "data/test_mask_tiles", 256, 256)


In [ ]:
import os

In [ ]:
os.getcwd()

In [ ]:
import os

input_dir = "data/image-tiles/"
target_dir = "data/mask-tiles/"
img_size = (256, 256)
num_classes = 2
batch_size = 32

input_img_paths = sorted([os.path.join(input_dir, fname) for fname in os.listdir(input_dir)])
target_img_paths = sorted([os.path.join(target_dir, fname) for fname in os.listdir(target_dir)])


In [ ]:
print("Number of samples:", len(input_img_paths))

for input_path, target_path in zip(input_img_paths[:10], target_img_paths[:10]):
    print(input_path, "|", target_path)

In [ ]:
from IPython.display import Image, display
from keras.utils import load_img
from PIL import ImageOps

# Display input image #7
display(Image(filename=input_img_paths[7]))
print(input_img_paths[7])
# Display auto-contrast version of corresponding target (per-pixel categories)
img = ImageOps.autocontrast(load_img(target_img_paths[7]))
display(img)

In [ ]:
import matplotlib.pyplot as plt
import PIL.Image as pim
import cv2

In [ ]:
# img = Image(filename=input_img_paths[7])
# img = pim.open(input_img_paths[7], "r")
# mask = pim.open(target_img_paths[7])
img = cv2.imread(input_img_paths[7], cv2.IMREAD_COLOR)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
plt.imshow(img)
plt.axis('off')
plt.show()

In [ ]:
import PIL
import numpy as np

In [ ]:
start = 300
count = 20
imgs = range(start, start + count)
imgs = [101, 103, 112, 115, 19, 307]
fig, axs = plt.subplots(2, len(imgs), figsize=(10, 3))
img = ImageOps.autocontrast(load_img(target_img_paths[7]))
x = 0
for img_index in imgs:
    img = cv2.imread(input_img_paths[img_index], cv2.IMREAD_COLOR)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    # mask = cv2.imread(target_img_paths[img_index], cv2.IMREAD_COLOR)
    mask = ImageOps.autocontrast(load_img(target_img_paths[img_index]))
    mask = np.array(mask)
    axs[0, x].imshow(img)
    axs[0, x].get_xaxis().set_visible(False)
    axs[0, x].get_yaxis().set_visible(False)
    axs[1, x].imshow(mask)
    axs[1, x].get_xaxis().set_visible(False)
    axs[1, x].get_yaxis().set_visible(False)
    x += 1
plt.show()

In [ ]:
plt.rcParams["figure.figsize"] = (30,30)

In [ ]:
img = cv2.imread("data/images-annotated/1.jpg", cv2.IMREAD_COLOR)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
mask = cv2.imread("data/masks/1.png")
mask = cv2.cvtColor(mask, cv2.COLOR_BGR2RGB)
#mask = (((mask * 255)/2)+100).astype(np.uint8)
# mask = (mask + 1)/3
mask = mask.astype(np.float32)
img = img.astype(np.float32)
mask[mask == 0] = .5
res = img * mask
res = res.astype(np.uint8)
plt.imshow(res)
plt.axis('off')
plt.show()

In [ ]:
import keras
import numpy as np
from tensorflow import data as tf_data
from tensorflow import image as tf_image
from tensorflow import io as tf_io


def get_dataset(
    batch_size,
    img_size,
    input_img_paths,
    target_img_paths,
    max_dataset_len=None,
):
    """Returns a TF Dataset."""

    def load_img_masks(input_img_path, target_img_path):
        input_img = tf_io.read_file(input_img_path)
        input_img = tf_io.decode_png(input_img, channels=3)
        input_img = tf_image.resize(input_img, img_size)
        input_img = tf_image.convert_image_dtype(input_img, "float32")

        target_img = tf_io.read_file(target_img_path)
        target_img = tf_io.decode_png(target_img, channels=1)
        target_img = tf_image.resize(target_img, img_size, method="nearest")
        target_img = tf_image.convert_image_dtype(target_img, "uint8")

        # Ground truth labels are 1, 2, 3. Subtract one to make them 0, 1, 2:
        # target_img -= 1
        return input_img, target_img

    # For faster debugging, limit the size of data
    if max_dataset_len:
        input_img_paths = input_img_paths[:max_dataset_len]
        target_img_paths = target_img_paths[:max_dataset_len]
    dataset = tf_data.Dataset.from_tensor_slices((input_img_paths, target_img_paths))
    dataset = dataset.map(load_img_masks, num_parallel_calls=tf_data.AUTOTUNE)
    return dataset.batch(batch_size)

In [ ]:
from keras import layers


def get_model(img_size, num_classes):
    inputs = keras.Input(shape=img_size + (3,))

    ### [First half of the network: downsampling inputs] ###

    # Entry block
    x = layers.Conv2D(32, 3, strides=2, padding="same")(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    previous_block_activation = x  # Set aside residual

    # Blocks 1, 2, 3 are identical apart from the feature depth.
    for filters in [64, 128, 256]:
        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(filters, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(filters, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.MaxPooling2D(3, strides=2, padding="same")(x)

        # Project residual
        residual = layers.Conv2D(filters, 1, strides=2, padding="same")(
            previous_block_activation
        )
        x = layers.add([x, residual])  # Add back residual
        previous_block_activation = x  # Set aside next residual

    ### [Second half of the network: upsampling inputs] ###

    for filters in [256, 128, 64, 32]:
        x = layers.Activation("relu")(x)
        x = layers.Conv2DTranspose(filters, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.Activation("relu")(x)
        x = layers.Conv2DTranspose(filters, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        # x = layers.UpSampling2D(2)(x)
        x = layers.UpSampling2D(2, interpolation='nearest')(x)

        # Project residual
        residual = layers.UpSampling2D(2, interpolation='nearest')(previous_block_activation)
        residual = layers.Conv2D(filters, 1, padding="same")(residual)
        # residual = layers.UpSampling2D(2)(previous_block_activation)
        # residual = layers.Conv2D(filters, 1, padding="same")(residual)
        
        x = layers.add([x, residual])  # Add back residual
        previous_block_activation = x  # Set aside next residual

    # Add a per-pixel classification layer
    outputs = layers.Conv2D(num_classes, 3, activation="softmax", padding="same")(x)

    # Define the model
    model = keras.Model(inputs, outputs)
    return model


# Build model
model = get_model(img_size, num_classes)
model.summary()

In [ ]:
import random

# Split our img paths into a training and a validation set
val_samples = 1000
random.Random(1337).shuffle(input_img_paths)
random.Random(1337).shuffle(target_img_paths)
train_input_img_paths = input_img_paths[:-val_samples]
train_target_img_paths = target_img_paths[:-val_samples]
val_input_img_paths = input_img_paths[-val_samples:]
val_target_img_paths = target_img_paths[-val_samples:]

# Instantiate dataset for each split
# Limit input files in `max_dataset_len` for faster epoch training time.
# Remove the `max_dataset_len` arg when running with full dataset.
train_dataset = get_dataset(
    batch_size,
    img_size,
    train_input_img_paths,
    train_target_img_paths,
    max_dataset_len=1000,
)
valid_dataset = get_dataset(
    batch_size, img_size, val_input_img_paths, val_target_img_paths
)

In [ ]:
import tensorflow as tf
print("Is GPU available:", tf.config.list_physical_devices('GPU'))
print("TensorFlow version:", tf.__version__)
print("CUDA version:", tf.sysconfig.get_build_info()["cuda_version"])
print("cuDNN version:", tf.sysconfig.get_build_info()["cudnn_version"])

In [ ]:
# Configure the model for training.
# We use the "sparse" version of categorical_crossentropy
# because our target data is integers.
model.compile(
    optimizer=keras.optimizers.Adam(1e-4), loss="sparse_categorical_crossentropy"
)

callbacks = [
    keras.callbacks.ModelCheckpoint("oxford_segmentation.tf", save_best_only=True)
]

# Train the model, doing validation at the end of each epoch.
epochs = 50
model.fit(
    train_dataset,
    epochs=epochs,
    validation_data=valid_dataset,
    callbacks=callbacks,
    verbose=2,
)

In [ ]:
# Generate predictions for all images in the validation set

val_dataset = get_dataset(
    batch_size, img_size, val_input_img_paths, val_target_img_paths
)
val_preds = model.predict(val_dataset)


def display_mask(i):
    """Quick utility to display a model's prediction."""
    mask = np.argmax(val_preds[i], axis=-1)
    mask = np.expand_dims(mask, axis=-1)
    img = ImageOps.autocontrast(keras.utils.array_to_img(mask))
    display(img)




In [ ]:
len(val_preds)

In [ ]:
# Display results for validation image #10


# Display mask predicted by our model
# display_mask(i)  # Note that the model only sees inputs at 150x150.

In [ ]:
# Display results for validation image #10
i = 16

# Display input image
display(Image(filename=val_input_img_paths[i]))
    # Display ground-truth target mask
img = ImageOps.autocontrast(load_img(val_target_img_paths[i]))
display(img)

# Display mask predicted by our model
display_mask(i)  # Note that the model only sees inputs at 150x150.

In [ ]:
masker.create_masks_for_folder("data/source-images-test", "data/source-annotations-test", "data/source-masks-test")

In [ ]:
annotator.create_image_annotations_for_folder("data/source-images-test", "data/source-annotations-test", "data/source-images-annotated-test")

In [ ]:
resizer.resize_images_in_folder("data/source-images-test", "data/test-images", (5000, 5000))

In [ ]:
resizer.resize_images_in_folder("data/source-masks-test", "data/test-masks", (5000, 5000))

In [ ]:
resizer.resize_images_in_folder("data/source-images-annotated-test", "data/images-annotated-test", (5000, 5000))

In [ ]:
tiler.split_to_tiles("data/source-images-test", "data/test-tiles", "data/source-masks-test", "data/test_mask-tiles", 256, 256 ,"data/visualized_masks/tile.csv", "")

In [ ]:
import tensorflow as tf
from PIL import ImageOps
import keras
import numpy as np
from tensorflow import data as tf_data
from tensorflow import image as tf_image
from tensorflow import io as tf_io
img_size = (256, 256)
num_classes = 2
batch_size = 32
def get_dataset(
    batch_size,
    img_size,
    input_img_paths,
    target_img_paths,
    max_dataset_len=None,
):
    """Returns a TF Dataset."""

    def load_img_masks(input_img_path, target_img_path):
        input_img = tf_io.read_file(input_img_path)
        input_img = tf_io.decode_png(input_img, channels=3)
        input_img = tf_image.resize(input_img, img_size)
        input_img = tf_image.convert_image_dtype(input_img, "float32")

        target_img = tf_io.read_file(target_img_path)
        target_img = tf_io.decode_png(target_img, channels=1)
        target_img = tf_image.resize(target_img, img_size, method="nearest")
        target_img = tf_image.convert_image_dtype(target_img, "uint8")

        # Ground truth labels are 1, 2, 3. Subtract one to make them 0, 1, 2:
        # target_img -= 1
        return input_img, target_img

    # For faster debugging, limit the size of data
    if max_dataset_len:
        input_img_paths = input_img_paths[:max_dataset_len]
        target_img_paths = target_img_paths[:max_dataset_len]
    dataset = tf_data.Dataset.from_tensor_slices((input_img_paths, target_img_paths))
    dataset = dataset.map(load_img_masks, num_parallel_calls=tf_data.AUTOTUNE)
    return dataset.batch(batch_size)
model = tf.keras.models.load_model('oxford_segmentation.tf')

In [ ]:
import os
import re
import tools.visualise as visualizer
input_img_paths = sorted([os.path.join("data/test_tiles", fname) for fname in os.listdir("data/test_tiles")])
target_img_paths = sorted([os.path.join("data/test_mask_tiles", fname) for fname in os.listdir("data/test_mask_tiles")])
input_img_paths = sorted(input_img_paths, key=lambda x: [int(i) if i.isdigit() else i for i in re.split('(\d+)', x)])
target_img_paths = sorted(target_img_paths, key=lambda x: [int(i) if i.isdigit() else i for i in re.split('(\d+)', x)])


In [ ]:
#test_dataset = get_dataset(
#    16, img_size, input_img_paths,target_img_paths
#)

#test_preds = model.predict(test_dataset)

In [ ]:
import os
def find_images_with_prefix(base_path, prefix):
    image_set = set()
    for fname in os.listdir(base_path):
        if fname.startswith(prefix):
            image_set.add(os.path.join(base_path, fname))
    sorted_images = sorted(image_set, key=lambda x: [int(i) if i.isdigit() else i for i in re.split('(\d+)', x)])
    return sorted_images
    
base_path = "data/test-tiles"
base_mask_path = "data/test_mask-tiles"
prefix_45 = "45."
prefix_79 = "79."
prefix_43 = "43."
prefix_91 = "91."
input_img_paths_45 = find_images_with_prefix(base_path, prefix_45)
target_img_paths_45 = find_images_with_prefix(base_mask_path, prefix_45)
input_img_paths_79 = find_images_with_prefix(base_path, prefix_79)
target_img_paths_79 = find_images_with_prefix(base_mask_path, prefix_79)
input_img_paths_43 = find_images_with_prefix(base_path, prefix_43)
target_img_paths_43 = find_images_with_prefix(base_mask_path, prefix_43)
input_img_paths_91 = find_images_with_prefix(base_path, prefix_91)
target_img_paths_91 = find_images_with_prefix(base_mask_path, prefix_91)

In [ ]:
test_dataset_45 = get_dataset(
    16, img_size, input_img_paths_45,target_img_paths_45
)

test_preds_45 = model.predict(test_dataset_45)

test_dataset_79 = get_dataset(
    16, img_size, input_img_paths_79,target_img_paths_79
)

test_preds_79 = model.predict(test_dataset_79)

test_dataset_43 = get_dataset(
    16, img_size, input_img_paths_43, target_img_paths_43
)

test_preds_43 = model.predict(test_dataset_43)

test_dataset_91 = get_dataset(
    16, img_size, input_img_paths_91,target_img_paths_91
)

test_preds_91 = model.predict(test_dataset_91)

In [ ]:
test_pred_list = []
def generate_pred_list(i):
    mask = np.argmax(test_preds_45[i], axis=-1)
    mask = np.expand_dims(mask, axis=-1)
    img = ImageOps.autocontrast(keras.utils.array_to_img(mask))
    test_pred_list.append(img)

for i in range(len(test_preds_45)):
    generate_pred_list(i)

In [ ]:
gd_rect_45 = visualizer.overlay_gd_and_rectangles(mask_dir_path="data/source-masks-test/", file_path="data/visualized_masks/tile.csv", image_number=45, dir_path="data/source-images-test/", save_path="data/visualized_masks/rectangles_" ,mask_save_path="data/visualized_masks/merged_rectangles_with_mask_" )

In [ ]:
pred_45 = visualizer.merge_prediction_csv("data/visualized_masks/tile.csv", test_pred_list, input_img_paths_45, "data/source-images-test/", 45, "data/visualized_masks/prediction_mask_", "data/source-images-test/")

In [ ]:
final_mask_45 = visualizer.final_mask_overlay("data/visualized_masks/prediction_mask_",  45,   "data/visualized_masks/merged_rectangles_with_mask_", "data/visualized_masks/final_mask_")

In [ ]:
test_pred_list = []
def generate_pred_list(i):
    mask = np.argmax(test_preds_79[i], axis=-1)
    mask = np.expand_dims(mask, axis=-1)
    img = ImageOps.autocontrast(keras.utils.array_to_img(mask))
    test_pred_list.append(img)

for i in range(len(test_preds_79)):
    generate_pred_list(i)

In [ ]:
gd_rect_79 = visualizer.overlay_gd_and_rectangles("data/source-masks-test/", "data/visualized_masks/tile.csv", 79, "data/source-images-test/", "data/visualized_masks/rectangles_" ,"data/visualized_masks/merged_rectangles_with_mask_" )

In [ ]:
pred_79 = visualizer.merge_prediction_csv("data/visualized_masks/tile.csv", test_pred_list, input_img_paths_79, "data/source-images-test/", 79, "data/visualized_masks/prediction_mask_", "data/source-images-test/")

In [ ]:
final_mask_79 = visualizer.final_mask_overlay("data/visualized_masks/prediction_mask_", 79, "data/visualized_masks/merged_rectangles_with_mask_",  "data/visualized_masks/final_mask_")

In [ ]:
test_pred_list = []
def generate_pred_list(i):
    mask = np.argmax(test_preds_43[i], axis=-1)
    mask = np.expand_dims(mask, axis=-1)
    img = ImageOps.autocontrast(keras.utils.array_to_img(mask))
    test_pred_list.append(img)

for i in range(len(test_preds_43)):
    generate_pred_list(i)

In [ ]:
gd_rect_43 = visualizer.overlay_gd_and_rectangles("data/source-masks-test/", "data/visualized_masks/tile.csv", 43, "data/source-images-test/", "data/visualized_masks/rectangles_" ,"data/visualized_masks/merged_rectangles_with_mask_" )

In [ ]:
pred_43 = visualizer.merge_prediction_csv("data/visualized_masks/tile.csv", test_pred_list, input_img_paths_43, "data/source-images-test/", 43, "data/visualized_masks/prediction_mask_", "data/source-images-test/")

In [ ]:
final_mask_43 = visualizer.final_mask_overlay("data/visualized_masks/prediction_mask_", 43,"data/visualized_masks/merged_rectangles_with_mask_",  "data/visualized_masks/final_mask_")

In [ ]:
test_pred_list = []
def generate_pred_list(i):
    mask = np.argmax(test_preds_91[i], axis=-1)
    mask = np.expand_dims(mask, axis=-1)
    img = ImageOps.autocontrast(keras.utils.array_to_img(mask))
    test_pred_list.append(img)

for i in range(len(test_preds_91)):
    generate_pred_list(i)

In [ ]:
gd_rect_91 = visualizer.overlay_gd_and_rectangles("data/source-masks-test/", "data/visualized_masks/tile.csv", 91, "data/source-images-test/", "data/visualized_masks/rectangles_" ,"data/visualized_masks/merged_rectangles_with_mask_" )

In [ ]:
pred_91 = visualizer.merge_prediction_csv("data/visualized_masks/tile.csv", test_pred_list, input_img_paths_91, "data/source-images-test/", 91, "data/visualized_masks/prediction_mask_", "data/source-images-test/")

In [ ]:
final_mask_91 = visualizer.final_mask_overlay("data/visualized_masks/prediction_mask_", 91,"data/visualized_masks/merged_rectangles_with_mask_",  "data/visualized_masks/final_mask_")